## HW Assignment 2

In this assignment, we will learn how to use Apache Spark DataFrames and the MLLib package used for machine learning.
Run the code below to start up a local Spark instance.

## Name
Robert Thompson (rt598@drexel.edu)

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Spark 2.4.5 not found at original link. Using 3.0.3 from apache archive
!wget https://archive.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar xf spark-3.0.3-bin-hadoop2.7.tgz

--2023-03-11 22:55:17--  https://archive.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 220400553 (210M) [application/x-gzip]
Saving to: ‘spark-3.0.3-bin-hadoop2.7.tgz’

spark-3.0.3-bin-had 100%[===================>] 210.19M  24.7MB/s    in 9.2s    

2023-03-11 22:55:27 (22.9 MB/s) - ‘spark-3.0.3-bin-hadoop2.7.tgz’ saved [220400553/220400553]



In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.0.3-bin-hadoop2.7"

In [3]:
!pip install -q findspark
# Explicitly define version here
!pip install pyspark==3.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.1/209.1 MB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 KB 22.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.3-py2.py3-none-any.whl size=209435969 sha256=56e0c02d5f7bbfb2a729a8a0e2cdd094ff2a1bc488d25e77dc9a6dcdc3ee1f96
  Stored in directory: /root/.cache/pip/wheels/65/ff/a2/0e26ceacea69c69610bbbd569678580b54be2e6e8b88e0eb9a
Successfully built pyspark


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pyspark.sql import SparkSession

In [6]:
APP_NAME = "HW2"

In [7]:
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()

In [8]:
spark

1. a. In the first part of this assignment, we will load a dataset and discover some insight about the data.
Load the travel dataset provided in the assignment with the option inferSchema set to true. Print 20 rows from the table

In [9]:
should_infer_schema = "true"
contains_header = "true"
delimiter = ","
file_type = "csv"
file_name = "drive/MyDrive/travel insurance.csv"

dataframe = spark.read.format(file_type) \
  .option("inferSchema", should_infer_schema) \
  .option("header", contains_header) \
  .option("sep", delimiter) \
  .csv(file_name)

In [10]:
dataframe.show(20)

+------+-------------+--------------------+--------------------+-----+--------+--------------------+---------+--------------------+------+---+
|Agency|  Agency Type|Distribution Channel|        Product Name|Claim|Duration|         Destination|Net Sales|Commision (in value)|Gender|Age|
+------+-------------+--------------------+--------------------+-----+--------+--------------------+---------+--------------------+------+---+
|   CBH|Travel Agency|             Offline|  Comprehensive Plan|   No|     186|            MALAYSIA|    -29.0|                9.57|     F| 81|
|   CBH|Travel Agency|             Offline|  Comprehensive Plan|   No|     186|            MALAYSIA|    -29.0|                9.57|     F| 71|
|   CWT|Travel Agency|              Online|Rental Vehicle Ex...|   No|      65|           AUSTRALIA|    -49.5|                29.7|  null| 32|
|   CWT|Travel Agency|              Online|Rental Vehicle Ex...|   No|      60|           AUSTRALIA|    -39.6|               23.76|  null| 32|

b. Rename the Commision (in value) column to Commission. Assign this dataframe to a new variable.

In [11]:
dataframe2 = dataframe.withColumnRenamed("Commision (in value)", "Commission")
dataframe2.show(1)

+------+-------------+--------------------+------------------+-----+--------+-----------+---------+----------+------+---+
|Agency|  Agency Type|Distribution Channel|      Product Name|Claim|Duration|Destination|Net Sales|Commission|Gender|Age|
+------+-------------+--------------------+------------------+-----+--------+-----------+---------+----------+------+---+
|   CBH|Travel Agency|             Offline|Comprehensive Plan|   No|     186|   MALAYSIA|    -29.0|      9.57|     F| 81|
+------+-------------+--------------------+------------------+-----+--------+-----------+---------+----------+------+---+
only showing top 1 row



c. Compute the count of policies for each destination. Print the top 10 destinations by the count of policies.

In [12]:
dataframe2.groupby("Destination").count().sort("count", ascending=False).show(10)

+-------------+-----+
|  Destination|count|
+-------------+-----+
|    SINGAPORE|13255|
|     MALAYSIA| 5930|
|     THAILAND| 5894|
|        CHINA| 4796|
|    AUSTRALIA| 3694|
|    INDONESIA| 3452|
|UNITED STATES| 2530|
|  PHILIPPINES| 2490|
|    HONG KONG| 2411|
|        INDIA| 2251|
+-------------+-----+
only showing top 10 rows



d. What is the mean age for customers who filed a claim? What is the mean age for customers who did not file a claim?
Print the mean age for both customers who filed and those who didn't file.

In [13]:
from pyspark.sql.functions import avg, col

Average Age of Customer Who Filed a Claim

In [14]:
dataframe2.filter(dataframe2["Claim"] == "Yes").agg(avg(col("age"))).show()

+-----------------+
|         avg(age)|
+-----------------+
|38.63430420711974|
+-----------------+



Average Age of Customers Who Did NOT File a Claim

In [15]:
dataframe2.filter(dataframe2["Claim"] == "No").agg(avg(col("age"))).show()

+------------------+
|          avg(age)|
+------------------+
|39.989823554864664|
+------------------+



e. Which travel agency made the most amount of money in commission? Compute the total amount of commission for each agency and print the top 10 agencies ordered by the amount.

In [16]:
from pyspark.sql.functions import sum

dataframe2.groupby("Agency").sum("Commission").sort("sum(Commission)", ascending=False).show(10)

+------+------------------+
|Agency|   sum(Commission)|
+------+------------------+
|   CWT|277825.68000001844|
|   C2B|169747.34000000358|
|   JZI| 74471.24999999916|
|   LWC| 51169.12999999995|
|   JWT|16208.399999999956|
|   KML| 8550.380000000014|
|   TST|           5556.25|
|   RAB| 5239.199999999985|
|   ART|3493.3499999999995|
|   ADM| 3136.899999999999|
+------+------------------+
only showing top 10 rows



2. a. The second part of the assignment will be to create a transformation pipeline. Remove all rows with missing data, convert all strings to integers, create dummy variables and assemble a feature vector. Use only the following variables as predictors: Agency Type, Distribution Channel, Duration, Net Sales, Commission, Gender, Age. Use Claim as the response variable.

In [17]:
dataframe2.show(10)

+------+-------------+--------------------+--------------------+-----+--------+-------------+---------+----------+------+---+
|Agency|  Agency Type|Distribution Channel|        Product Name|Claim|Duration|  Destination|Net Sales|Commission|Gender|Age|
+------+-------------+--------------------+--------------------+-----+--------+-------------+---------+----------+------+---+
|   CBH|Travel Agency|             Offline|  Comprehensive Plan|   No|     186|     MALAYSIA|    -29.0|      9.57|     F| 81|
|   CBH|Travel Agency|             Offline|  Comprehensive Plan|   No|     186|     MALAYSIA|    -29.0|      9.57|     F| 71|
|   CWT|Travel Agency|              Online|Rental Vehicle Ex...|   No|      65|    AUSTRALIA|    -49.5|      29.7|  null| 32|
|   CWT|Travel Agency|              Online|Rental Vehicle Ex...|   No|      60|    AUSTRALIA|    -39.6|     23.76|  null| 32|
|   CWT|Travel Agency|              Online|Rental Vehicle Ex...|   No|      79|        ITALY|    -19.8|     11.88|  nu

In [18]:
# Remove all rows with missing data
dataframe3 = dataframe2.na.drop()
dataframe3.show(10)

+------+-------------+--------------------+------------------+-----+--------+-------------+---------+----------+------+---+
|Agency|  Agency Type|Distribution Channel|      Product Name|Claim|Duration|  Destination|Net Sales|Commission|Gender|Age|
+------+-------------+--------------------+------------------+-----+--------+-------------+---------+----------+------+---+
|   CBH|Travel Agency|             Offline|Comprehensive Plan|   No|     186|     MALAYSIA|    -29.0|      9.57|     F| 81|
|   CBH|Travel Agency|             Offline|Comprehensive Plan|   No|     186|     MALAYSIA|    -29.0|      9.57|     F| 71|
|   JZI|     Airlines|              Online|        Value Plan|   No|      66|UNITED STATES|   -121.0|     42.35|     F| 44|
|   JZI|     Airlines|              Online|        Basic Plan|   No|       1|     MALAYSIA|    -18.0|       6.3|     M| 47|
|   KML|Travel Agency|              Online|      Premier Plan|   No|      53|       NORWAY|   -130.0|      49.4|     F| 48|
|   JZI|

In [19]:
from pyspark.ml.feature import StringIndexer

# Convert all strings to integers by adding a new column
columns = ["Agency Type", "Distribution Channel", "Claim", "Gender"]
stages = [StringIndexer(inputCol=column, outputCol=column + "_Index").fit(dataframe3) for column in columns]

In [20]:
from pyspark.ml.feature import VectorAssembler

# Use only the following variables as predictors
feature_columns = ["Agency Type_Index", "Distribution Channel_Index", "Gender_Index", "Duration", "Net Sales", "Commission", "Age"]
# Assemble a feature vector
stages.append(VectorAssembler(inputCols=feature_columns, outputCol='features'))

In [21]:
from pyspark.ml import Pipeline

# Create transformation pipeline
pipeline = Pipeline(stages=stages)
dataframe4_transformed = pipeline.fit(dataset=dataframe3).transform(dataframe3)

In [22]:
dataframe4_transformed.show(5, truncate=False)

+------+-------------+--------------------+------------------+-----+--------+-------------+---------+----------+------+---+-----------------+--------------------------+-----------+------------+------------------------------------+
|Agency|Agency Type  |Distribution Channel|Product Name      |Claim|Duration|Destination  |Net Sales|Commission|Gender|Age|Agency Type_Index|Distribution Channel_Index|Claim_Index|Gender_Index|features                            |
+------+-------------+--------------------+------------------+-----+--------+-------------+---------+----------+------+---+-----------------+--------------------------+-----------+------------+------------------------------------+
|CBH   |Travel Agency|Offline             |Comprehensive Plan|No   |186     |MALAYSIA     |-29.0    |9.57      |F     |81 |1.0              |1.0                       |0.0        |1.0         |[1.0,1.0,1.0,186.0,-29.0,9.57,81.0] |
|CBH   |Travel Agency|Offline             |Comprehensive Plan|No   |186     

In [23]:
feature_data = dataframe4_transformed.withColumnRenamed("Claim_Index", "label").select(["label", "features"])
feature_data.show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|[1.0,1.0,1.0,186....|
|  0.0|[1.0,1.0,1.0,186....|
|  0.0|[0.0,0.0,1.0,66.0...|
|  0.0|[0.0,0.0,0.0,1.0,...|
|  0.0|[1.0,0.0,1.0,53.0...|
|  0.0|[0.0,0.0,0.0,3.0,...|
|  0.0|[0.0,0.0,1.0,12.0...|
|  0.0|[0.0,0.0,1.0,7.0,...|
|  1.0|[0.0,0.0,0.0,12.0...|
|  0.0|[0.0,0.0,0.0,190....|
+-----+--------------------+
only showing top 10 rows



b. Split the data into train and test with 20% of the data in the test sample.

In [24]:
seed = 10  # Set seed for reproducibility
train, test = feature_data.randomSplit([0.8, 0.2], seed=seed)
print("Total Dataset Count:", feature_data.count())
print("Train Dataset Count:", train.count())
print("Test Dataset Count:", test.count())

Total Dataset Count: 18219
Train Dataset Count: 14527
Test Dataset Count: 3692


In [25]:
train.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(7,[2,5,6],[1.0,5...|
|  0.0|(7,[3,5,6],[1.0,6...|
|  0.0|(7,[3,5,6],[2.0,4...|
|  0.0|(7,[3,5,6],[2.0,6...|
|  0.0|(7,[3,5,6],[3.0,1...|
+-----+--------------------+
only showing top 5 rows



In [26]:
test.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(7,[3,5,6],[1.0,6...|
|  0.0|(7,[3,5,6],[1.0,7...|
|  0.0|(7,[3,5,6],[3.0,6...|
|  0.0|(7,[3,5,6],[4.0,7...|
|  0.0|(7,[3,5,6],[5.0,7...|
+-----+--------------------+
only showing top 5 rows



c. Apply a logistic regression model to predict the probaility that a customer will make a claim. Use the training data to produce a model and then test it using the test dataset.

In [27]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import numpy as np

lr = LogisticRegression(featuresCol= "features", labelCol = "label", maxIter = 10)

paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, np.linspace(0.3, 0.01, 10)) \
    .addGrid(lr.elasticNetParam, np.linspace(0.3, 0.8, 6)) \
    .build()
crossval_lr = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid_lr,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds= 5)  
cvModel_lr = crossval_lr.fit(train)
best_model_lr = cvModel_lr.bestModel.summary

In [28]:
best_model_lr.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(7,[2,5,6],[1.0,5...|[3.66722348831959...|[0.97508910232146...|       0.0|
|  0.0|(7,[3,5,6],[1.0,6...|[3.57298459716901...|[0.97269457141352...|       0.0|
|  0.0|(7,[3,5,6],[2.0,4...|[3.52832897864109...|[0.97148315540013...|       0.0|
|  0.0|(7,[3,5,6],[2.0,6...|[3.57791225126375...|[0.97282514472307...|       0.0|
|  0.0|(7,[3,5,6],[3.0,1...|[3.64895093552205...|[0.97464138146449...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [29]:
train_preds_lr = cvModel_lr.transform(train)
train_preds_lr.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(7,[2,5,6],[1.0,5...|[3.66722348831959...|[0.97508910232146...|       0.0|
|  0.0|(7,[3,5,6],[1.0,6...|[3.57298459716901...|[0.97269457141352...|       0.0|
|  0.0|(7,[3,5,6],[2.0,4...|[3.52832897864109...|[0.97148315540013...|       0.0|
|  0.0|(7,[3,5,6],[2.0,6...|[3.57791225126375...|[0.97282514472307...|       0.0|
|  0.0|(7,[3,5,6],[3.0,1...|[3.64895093552205...|[0.97464138146449...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [30]:
test_preds_lr = cvModel_lr.transform(test)
test_preds_lr.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(7,[3,5,6],[1.0,6...|[3.55094758711449...|[0.97210313497934...|       0.0|
|  0.0|(7,[3,5,6],[1.0,7...|[3.63358637481893...|[0.97425885565383...|       0.0|
|  0.0|(7,[3,5,6],[3.0,6...|[3.70955271317197...|[0.97609687682082...|       0.0|
|  0.0|(7,[3,5,6],[4.0,7...|[3.77508214491664...|[0.97757902318057...|       0.0|
|  0.0|(7,[3,5,6],[5.0,7...|[3.55413044595259...|[0.97218932017245...|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [31]:
test_preds_lr.groupBy("label", "prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       0.0|  123|
|  0.0|       0.0| 3569|
+-----+----------+-----+



d. Compute the accuracy for the train and test datasets.

In [32]:
train_accuracy = train_preds_lr.filter(train_preds_lr.label == train_preds_lr.prediction).count() / float(train_preds_lr.count())
print("Train Accuracy:", train_accuracy)

Train Accuracy: 0.9644799339161562


In [33]:
test_accuracy = test_preds_lr.filter(test_preds_lr.label == test_preds_lr.prediction).count() / float(test_preds_lr.count())
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9666847237269772
